In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier

from urllib.request import Request, urlopen  # Python 3
import json
import os.path

from datetime import datetime

In [18]:
# Load the data sets
data_dir = './data/'
df_Trip = pd.read_csv(data_dir + 'Trip_Household_Merged.csv')    
df_Trip = df_Trip[(df_Trip["origin_lat"] < 48) & (df_Trip["origin_lat"] > 47)]
df_Trip = df_Trip[(df_Trip["dest_lat"] < 48) & (df_Trip["dest_lat"] > 47)]
df_Trip = df_Trip[(df_Trip["dest_lng"] > -123) & (df_Trip["dest_lng"] < -122)]
df_Trip = df_Trip[(df_Trip["origin_lng"] > -123) & (df_Trip["origin_lng"] < -122)]
df_Trip['bg_origin'] = df_Trip['bg_origin'].astype(str)


df_Trip['depart_time_timestamp'] = pd.to_datetime(df_Trip['depart_time_timestamp'])
df_Trip['hour'] = df_Trip.depart_time_timestamp.dt.hour

#print (df_Trip)

df_Blockgroups = pd.read_csv(data_dir + 'SeattleCensusBlocksandNeighborhoodCorrelationFile.csv') 
df_Blockgroups['BLOCKGROUP'] = df_Blockgroups['BLOCKGROUP'].astype(str)

#print (df_Blockgroups)

#print (list(df_Trip.columns.values))
#print (df_Trip)

df_Merged= pd.merge(left=df_Trip, right=df_Blockgroups, how='inner', left_on='bg_dest', right_on='BLOCKGROUP')
#print (list(df_Merged.columns.values))

df_Merged['bg_pair'] = df_Merged['bg_origin'].astype(str) + "-" + df_Merged['bg_dest'].astype(str)

df_Merged_agg = df_Merged.groupby(['bg_pair'], as_index=False).agg({'bg_origin':['max'],'bg_dest':['max'],
                                                                   'CT_LAT':['max'],'CT_LON':['max']})
df_Merged_agg.columns = df_Merged_agg.columns.droplevel(level=1)
df_Merged_agg = df_Merged_agg.rename(index=str, columns={"CT_LAT": "lat_dest", "CT_LON": "lon_dest"})

df_Merged_agg= pd.merge(left=df_Merged_agg, right=df_Blockgroups, how='inner', left_on='bg_origin', right_on='BLOCKGROUP')
df_Merged_agg = df_Merged_agg.rename(index=str, columns={"CT_LAT": "lat_origin", "CT_LON": "lon_origin"})


df_Trip = pd.read_csv(data_dir + 'Trip_Household_Merged.csv')    


print (df_Merged_agg)

/Users/xx/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                        bg_pair     bg_origin       bg_dest  lat_dest  \
0     530330001001-530330001005  530330001001  530330001005  47.72293   
1     530330001001-530330007001  530330001001  530330007001  47.71744   
2     530330001001-530330041005  530330001001  530330041005  47.65915   
3     530330001001-530330053023  530330001001  530330053023  47.65576   
4     530330001001-530330067001  530330001001  530330067001  47.63238   
5     530330001002-530330001003  530330001002  530330001003  47.72281   
6     530330001002-530330007001  530330001002  530330007001  47.71744   
7     530330001002-530330018001  530330001002  530330018001  47.69420   
8     530330001002-530330018002  530330001002  530330018002  47.69222   
9     530330001002-530330093002  530330001002  530330093002  47.58946   
10    530330001003-530330001002  530330001003  530330001002  47.72932   
11    530330001003-530330001005  530330001003  530330001005  47.72293   
12    530330001003-530330002002  530330001003  5303